In [1]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
# from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


os.environ["WANDB_NOTEBOOK_NAME"] = "GRU - v8.0.1 Topaz.ipynb"

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [4]:
torch.__version__

'2.1.1'

In [5]:
data_file = './data/topaz_data_w_bsp_new5_nz.fth'
df = pd.read_feather(data_file)

In [6]:
df.BSP

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
391881   NaN
391882   NaN
391883   NaN
391884   NaN
391885   NaN
Name: BSP, Length: 391886, dtype: float64

In [7]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
import rnn_tools.model_saver
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW"]
states = ["NZ"]
# states = ['VIC', "SA"]
extra_tracks = ["Wentworth Park", "Wagga", "Temora"]
only_tracks = ["Bendigo", "Geelong", "Traralgon", "Ballarat", "Shepparton", "Warragul","Healesville"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/topaz_data_w_bsp_new4_small.fth'
# data_file = './data/topaz_data_w_bsp_new4.fth'
data_file = './data/topaz_data_w_bsp_new6.fth'
# data_file = './data/topaz_data_w_bsp_new2.fth'
# data_file = './data/topaz_data_w_bsp_new_w_price.fth'
# data_file = './data/topaz_data_w_bsp.fth'
# data_file = './data/gru_inputs_kitchen_sink_topaz.fth'
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,only_tracks=only_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='boosted_sftmin',v6=True,show_stats=False)
# raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,track_filter=extra_tracks,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2518990, 156)
59
Latest date = 2024-05-19 00:00:00
size after state filter (2518990, 157)
(389662, 158)
Index(['index', 'meetingDate', 'state', 'track_name', 'dist', 'raceId',
       'race_grade', 'race_num', 'box', 'rugNumber',
       ...
       'prev_BSP_price', 'prev_start_price', 'stats_topaz', 'dogid', 'raceid',
       'stats_cols', 'track_hash', 'stats', 'race_time', 'stats_cuda'],
      dtype='object', length=159)
state  track_name  
NZ     Addington       84479
       Ascot Park      14435
       Ashburton          94
       Cambridge       21483
       Forbury Park     6163
       Hatrick         30213
       Manawatu        28183
       Manukau         23933
       Taranaki          235
       Tokoroa           179
Name: count, dtype: int64
Latest date = 2024-05-16 00:00:00
stats = [ 0.00000000e+00  1.00000000e+00  8.00000000e+00  1.35416672e-01
  0.00000000e+00  0.00000000e+00  1.48281574e-01  0.00000000e+00
  0.00000000e+00  0

100%|██████████| 25/25 [00:00<00:00, 25019.71it/s]


race_ids=['397536987', '397536988', '397536989', '397536993', '397536992', '397536990', '397536991', '397536994', '397536998', '397536995', '397536996', '397536997', '397537055', '397537056', '397537057', '397537060', '397537058', '397537059', '397537065', '397537061', '397537062', '397537064', '397537063', '397624250', '397537066', '397624270', '397624269', '397624268', '397624265', '397624266', '397624267', '397624256', '397624253', '397624263', '397624259', '397624246', '397624212', '397624211', '397624339', '397624214', '397624213', '397624217', '397624215', '397624216', '397624218', '397624220', '397624221', '397624329', '397624219', '397624222', '397624340', '397624334', '397624330', '397624331', '397624326', '397624328', '397624327', '397624338', '397624337', '397624332', '397624333', '397624335', '397624336', '397624319', '397624325', '397624324', '397624323', '397624322', '397624321', '397624320', '397624316', '397624318', '397624317', '397624315', '397624314', '399450852', '3

c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:417: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)
c:\Users\Nick\Documents\GitHub\grvmodel\python\rnn_tools\rnn_classes.py:418: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.start_prob =  torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 27703, number of unique dogs = 4978
0          (Hatrick, 305)
1          (Hatrick, 305)
2          (Hatrick, 305)
3          (Hatrick, 305)
4          (Hatrick, 520)
               ...       
27698    (Addington, 295)
27699    (Addington, 295)
27700    (Addington, 645)
27701    (Addington, 520)
27702    (Addington, 295)
Length: 27703, dtype: object


In [8]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def load_matching_state_dict(model, state_dict):
    model_dict = model.state_dict()

    # Filter out unnecessary keys
    matching_keys = {k: v for k, v in state_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
    excluded_keys = set(state_dict.keys()) - set(matching_keys.keys())

    # Overwrite entries in the existing state dict
    model_dict.update(matching_keys)

    # Load the new state dict
    model.load_state_dict(model_dict)

    print("Loaded keys:")
    for key in matching_keys:
        print(key)

    print("\nExcluded keys:")
    for key in excluded_keys:
        print(key)

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      print(reg_stat_mask)
      stat_mask = [1]*27+53*[0]
      data_mask =  [1]*27+53*[1]

      stat_mask = [1]*20+39*[0]
      data_mask =  [1]*20+39*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      print(f"{data_mask=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)
      config['input_size'] = input_size


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-06-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding_testing(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        load_matching_state_dict(model, model_data['model_state_dict'])
        config['parent model'] = prev_model_file
        model = model.to(device)        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      try:
        return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      raceDB.create_hidden_states_dict_v2()
      rnn_tools.model_saver.model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [9]:
raceDB.batches_setup = False

In [10]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0003,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2022-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 450,
    'stat_list_dict': stat_list_dict,
}

In [11]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  
    race.loss_log = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-05-16 00:00:00
tensor(nan, device='cuda:0')
402888097
tensor(nan, device='cuda:0')
458549780
tensor(nan, device='cuda:0')
618363326


In [12]:
torch.cuda.empty_cache()   

In [13]:
x = torch.rand(10,8)
x.argmax(dim=-1)

tensor([4, 4, 6, 3, 7, 4, 5, 1, 3, 1])

In [14]:
for r in raceDB.racesDict.values():
    try:
        r.start_prob
    except:
        print(r.raceid)
        print(r.raw_prices)

In [15]:
for r in raceDB.racesDict.values():
    print(r.prices)
    print(all([x==np.nan for x in r.prices]))
    print(all([np.isnan(x) for x in r.prices]))
    break

[nan, nan, nan, nan, nan, nan, nan, nan]
False
True


In [16]:
for r in raceDB.racesDict.values():
    try:
        r.fin_prob
    except:
        print(r.raceid)
        print(r.raw_prices)
        r

In [17]:
for r in raceDB.racesDict.values():
    print(r.prices)
    print(all([x==np.nan for x in r.prices]))
    print(all([(np.isnan(x) or x==0) for x in r.prices]))
    print(r.fin_prob)
    break

[nan, nan, nan, nan, nan, nan, nan, nan]
False
True
tensor([1.4384e-01, 4.1362e-01, 3.0142e-02, 2.5772e-04, 1.7897e-01, 1.9740e-01,
        2.6088e-02, 9.6931e-03], device='cuda:0')


In [18]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
importlib.reload(rnn_tools.model_saver)
# s
model_pipeline(raceDB,config=wandb_config_static,sweep=False) 

# model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='VIC SA NSW-ancient-valley-726', prev_model_version = 280) 


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


{'hidden_size': 256, 'stats': "['dogAgeScaled', 'boxNumber', 'incomingGrade_val', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_incomingGrade_val_mean_1', 'dog_startPrice_mean_1', 'dog_distance_mean_1', 'dog_boxNumber_mean_1', 'dog_runTimeNorm_mean_1', 'dog_win_runTimeNorm_mean_1', 'dog_delta_win_runTimeNorm_mean_1', 'dog_place_mean_1', 'dog_resultMargin_mean_1', 'dog_finishingPlaceMovement_mean_1', 'dog_StartingPlaceMovement_mean_1', 'dog_averageSpeed_mean_1', 'dog_win_mean_1', 'trainer_incomingGrade_val_mean_365D', 'trainer_startPrice_mean_365D', 'trainer_distance_mean_365D', 'trainer_boxNumber_mean_365D', 'trainer_runTimeNorm_mean_365D', 'trainer_win_runTimeNorm_mean_365D', 'trainer_delta_win_runTimeNorm_mean_365D', 'trainer_place_mean_365D', 'trainer_resultMargin_mean_365D', 'trainer_finishingPlaceMovement_mean_365D', 'trainer_StartingPlaceMovement_mean_365D', 'trainer_averageSpeed_mean_365D', 'trainer_win_mean_365D', 'd

100%|██████████| 4978/4978 [00:08<00:00, 604.01it/s] 


[(datetime.date(2019, 1, 2), datetime.date(2020, 3, 27)), (datetime.date(2020, 3, 27), datetime.date(2021, 6, 20)), (datetime.date(2021, 6, 20), datetime.date(2021, 12, 31))]
2020-03-27
2021-06-20
2021-12-31
Train examples [7165, 6309, 2535]


100%|██████████| 4977/4977 [00:01<00:00, 3725.65it/s]
3it [00:08,  2.76s/it]


...

100%|█████████▉| 2496/2500 [6:11:37<00:28,  7.18s/it]

Quickpass time 1.753713099999004


100%|██████████| 2500/2500 [6:12:04<00:00,  8.93s/it]


ROI < 30,▆▄▅▆▅▃▅▅▁▅▃▆▆▇▆▆█▇▅█▇▆▇▅▆█▅█▇▇▇███▇▇▇▆▇▇
ROI < 30 2,▆▄▄▅▅▂▃▄▁▅▄▇▇▇▆▇█▇▅▇▆▆▇▅▆▇▄▇▆▆▆▇▇▆▆▅▆▅▅▆
accuracy,▁▁▁▇████████████████████████████████████
accuracy2,▁▁▁▇████████████████████████████████████
avg_win_price,███▄▄▃▃▃▂▃▂▂▁▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▁▂▂▁▂▂▁▂
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
favorite_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
flat_simple,▄▁▁▇▇▆▆▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
kelly roi,▁▂▂▅▆▅▅▆▄▆▆▇▇▇▇▆█▇▇█▇▇▇▆▇█▆█▇█▇▇██▇█▇▇▇█
kelly seq roi,▁▁▁▄▅▃▅▅▃▅▅▇▇▇▆▇█▇▆█▇▆▇▆▆█▅█▇▇▇▇██▇▇▇▇▇▇
kelly seq roi < 30,▁▁▁▄▅▃▅▅▃▅▅▇▇▇▆▇█▇▆█▇▆▇▆▆█▅█▇▇▇▇██▇▇▇▇▇▇


GRUNetv3_extra_embedding_testing(
  (gru): GRU(20, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2099, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(59, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (track_embedding): Embedding(1024, 50)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): Linear(in_features=59, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_features=20